In [1]:
import os
import errno
import glob
import shutil
import scipy.io as io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import re

# import location for specific functions
path_to_face_front_utilyties = '/home/gonzalo/M9/Methods/face-frontalization'
sys.path.append(path_to_face_front_utilyties)
import check_resources as check
import frontalize
import facial_feature_detector as feature_detection
import camera_calibration as calib

#Identify OS platform 
#print os.name
import platform
working_platform = platform.system()
working_release = platform.release()
print working_platform




Linux


In [2]:
##################################################################
# Subssidiary functions #
##################################################################

def make_sure_path_exists(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
        except OSError as error:
            if error.errno != errno.EEXIST:
                raise
                
def make_sure_file_exists(infile):
    if not os.path.isfile(infile):
        try:
            with open(infile, 'w') as f:
                print 'file found...'
        except OSError as error:
            if error.errno != errno.EEXIST:
                raise

def seek_or_write(infile, searched_path, personID):
    with open(infile, "r+") as f:
        line_found = any(searched_path in line for line in f)
        if not line_found:
            f.seek(0, os.SEEK_END)
            f.write('%s %s\n' % (searched_path, personID))
        else:
            print 'file already listed'

In [3]:
# This function implements Haar face cascade method for face detection as in:
# https://www.superdatascience.com/opencv-face-detection/
# With modifications to only output the biggest bounding box as the face detection 
# (works on this particular case due to the nature of the dataset used as there is only one face per image)
def detect_face(gray_img, haar_face_cascade):
    roi_size_old = 0
    list_counter = 0
    biggest_val_index = 0
    face_area = ()
    faces = haar_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5);
    #print the number of faces found 
    print 'Faces found: ', len(faces)
    print faces
    if len(faces)==0:
        print 'none faces found'
    elif len(faces)==1:
        face_area = faces[list_counter]
        img_w_face = gray_img[face_area[0]:face_area[0]+face_area[2], face_area[1]:face_area[1]+face_area[3]]
        return img_w_face
    else:
        for (x, y, w, h) in faces:
            #print (x, y, w, h)
            roi_size = int(w) * int(h)
            #print 'current roi',roi_size, ' and old roi =', roi_size_old
            if roi_size < roi_size_old:
                roi_size = roi_size_old
            #    print 'new roi_size_old = ', roi_size_old
                biggest_val_index = biggest_val_index
            #    print list_counter
            else:
                roi_size_old = roi_size
            #    print 'new roi_size_old = ', roi_size_old
                biggest_val_index += list_counter
            #    print list_counter
            list_counter += 1
            face_area = faces[biggest_val_index]          
            img_w_face = gray_img[face_area[0]:face_area[0]+face_area[2], face_area[1]:face_area[1]+face_area[3]]
        return img_w_face

In [18]:
def front_database_build(name, data_path, work_path, mode, eyemask, filelist):
    ''' 
    Given the name of the subject in the variable -name-, the path from where to obtain the images, 
    the path to the destiny folder in which to save the database and the eyemask parameter used for
    frontalization of the faces, this routine takes the images from data_path and saves the 
    frontalized face obtained in the work_path separating pictures in folders for each individual. 
    '''
    for x in name[:]:
        pathdir = os.path.join(work_path, mode, x)
    
        print pathdir #for debug purposes
        make_sure_path_exists(pathdir)
        search_filename = x +  "*.pgm" # "*.jpg" #
        search_location = os.path.join(data_path,search_filename)
        print search_location #for debug purposes
    
        for filepath in glob.glob(search_location):
            
            # Extract filename from the filepath found at search location
            tail = re.split(r"[.]", os.path.basename(filepath))
            tail = str(tail[0])
            print tail #for debug purposes
            out_name = tail + 'fr.jpg'
            
            # For MIT-CBCL dataset file naming, break name into the image data provided in filename
            # 'subject' - 'face rotation' - etc
            file_data = re.split(r"[_]", tail)
            #print file_data #for debug purposes
            
            # angular limit for rotated face frontalization
            face_rot = -20
            
            #Copy only the images that have less than face_rot rotation angle
            if os.path.isfile(filepath): #and face_rot<float(file_data[1]):
    
                image = cv2.imread(filepath,1)# cv2.IMREAD_GRAYSCALE)

                if image is not None:
                    
                    # extract landmarks from the query image
                    # list containing a 2D array with points (x, y) for each face detected in the query image
                    lmarks = feature_detection.get_landmarks(image)
                    if lmarks.any():
                        # perform camera calibration according to the first face detected
                        proj_matrix, camera_matrix, rmat, tvec = calib.estimate_camera(model3D, lmarks[0])

                        # perform frontalization
                        frontal_raw, frontal_sym = frontalize.frontalize(image, proj_matrix, model3D.ref_U, eyemask)
                        #write frontal_sym to image file in pathdir location
                        out_file = os.path.join(pathdir,out_name)
                        print out_file
                        if not os.path.isfile(out_file):
                            #turn image to gray scale
                            gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                            if gray_img is not None:
                                img = detect_face(gray_img, haar_face_cascade)
                                cv2.imwrite(out_file ,img)
                                print 'image converted...'
                            else:
                                print 'there is no image converted...'
                            print 'image created'
                        else:
                            print 'image already exists'
                        seek_or_write(filelist, out_file, x)

In [19]:
if __name__ == "__main__":
    
    mode = 'test'
    
    #Directories definition function
    #Directory to build processed dataset
    work_path = "/home/gonzalo/dev/post_data/MIT" #SCfaces"
    print work_path #for debug purposes
    make_sure_path_exists(work_path)
    #Directory to from where to extract data
    root_dir = "/home/gonzalo"
    make_sure_path_exists(root_dir)
    data_path = os.path.join(root_dir,"M9","Datasets","MIT-CBCL", 'test')#"training-synthetic") #,'SCface_database','mugshot_rotation_all') #
    print (data_path) #for debug purposes
    
    txt_file_list = os.path.join(work_path, mode,'list.txt')
    make_sure_file_exists(txt_file_list)
    
    #Set face detector
    if not os.path.isfile('/home/gonzalo/opencv-3.2.0/data/haarcascades/haarcascade_frontalface_alt.xml'):
        try:
            with open('/home/gonzalo/opencv-3.2.0/data/haarcascades/haarcascade_frontalface_alt.xml', 'r') as f:
                print 'haar_face_cascade loaded...'
        except OSError as error:
            if error.errno != errno.EEXIST:
                raise
    haar_face_cascade = cv2.CascadeClassifier('/home/gonzalo/opencv-3.2.0/data/haarcascades/haarcascade_frontalface_alt.xml')
    # check for dlib saved weights for face landmark detection
    # if it fails, dowload and extract it manually from
    # http://sourceforge.net/projects/dclib/files/dlib/v18.10/shape_predictor_68_face_landmarks.dat.bz2
    check.check_dlib_landmark_weights()
    # load detections performed by dlib library on 3D model and Reference Image
    model3D = frontalize.ThreeD_Model(path_to_face_front_utilyties + '/frontalization_models/model3Ddlib.mat', 'model_dlib')
    # load mask to exclude eyes from symmetry
    eyemask = np.asarray(io.loadmat(path_to_face_front_utilyties + '/frontalization_models/eyemask.mat')['eyemask'])
    
    # List of subjects numbers for folder creation
    name =["%04d" % x for x in range(10)] #for MIT-CBCL
    #name =["%03d" % x for x in range(1,131)] #for SCface
    #print name #for debug purposes
    
    # Launch frontalized faces dataset creation
    front_database_build(name, data_path, work_path, mode, eyemask, txt_file_list)
    print 'frontalization finished'
    

/home/gonzalo/dev/post_data/MIT
/home/gonzalo/M9/Datasets/MIT-CBCL/test
file found...
/home/gonzalo/dev/post_data/MIT/test/0000
/home/gonzalo/M9/Datasets/MIT-CBCL/test/0000*.pgm
0000_02225
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02225fr.jpg
Faces found:  1
[[19 17 81 81]]
image converted...
image created
0000_02732
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02732fr.jpg
Faces found:  1
[[19 18 79 79]]
image converted...
image created
0000_02967
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02967fr.jpg
Faces found:  1
[[23 20 78 78]]
image converted...
image created
0000_03522
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_03522fr.jpg
Faces found:  1
[[23 19 81 81]]
image converted...
image created
0000_05242
Number of faces detected: 1

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02614fr.jpg
Faces found:  1
[[16 19 79 79]]
image converted...
image created
0000_02244
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02244fr.jpg
Faces found:  1
[[19 18 81 81]]
image converted...
image created
0000_02471
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02471fr.jpg
Faces found:  1
[[16 19 78 78]]
image converted...
image created
0000_02997
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02997fr.jpg
Faces found:  1
[[10 20 75 75]]
image converted...
image created
0000_03508
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_03508fr.jpg
Faces found:  1
[[31 21 76 76]]
image converted...
image created
0000_02628
Number of faces dete

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02298fr.jpg
Faces found:  1
[[30 20 74 74]]
image converted...
image created
0000_02557
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02557fr.jpg
Faces found:  1
[[ 4 24 68 68]]
image converted...
image created
0000_02712
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02712fr.jpg
Faces found:  1
[[20 16 77 77]]
image converted...
image created
0000_03501
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_03501fr.jpg
Faces found:  1
[[33 22 74 74]]
image converted...
image created
0000_03375
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_03375fr.jpg
Faces found:  1
[[33 22 74 74]]
image converted...
image created
0000_02287
Number of faces dete

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_05194fr.jpg
Faces found:  1
[[29 24 72 72]]
image converted...
image created
0000_02632
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02632fr.jpg
Faces found:  1
[[19 18 80 80]]
image converted...
image created
0000_05147
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_05147fr.jpg
Faces found:  1
[[19 19 75 75]]
image converted...
image created
0000_03773
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_03773fr.jpg
Faces found:  1
[[16 17 83 83]]
image converted...
image created
0000_02316
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02316fr.jpg
Faces found:  1
[[37 22 71 71]]
image converted...
image created
0000_02281
Number of faces dete

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02291fr.jpg
Faces found:  1
[[30 21 75 75]]
image converted...
image created
0000_02827
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02827fr.jpg
Faces found:  1
[[23 19 80 80]]
image converted...
image created
0000_02558
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_02558fr.jpg
Faces found:  1
[[ 4 23 70 70]]
image converted...
image created
0000_03523
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_03523fr.jpg
Faces found:  1
[[22 19 81 81]]
image converted...
image created
0000_03528
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0000/0000_03528fr.jpg
Faces found:  1
[[21 19 81 81]]
image converted...
image created
0000_02597
Number of faces dete

...

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05011fr.jpg
Faces found:  1
[[26 10 66 66]]
image converted...
image created
0009_05028
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05028fr.jpg
Faces found:  1
[[27 10 68 68]]
image converted...
image created
0009_05206
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05206fr.jpg
Faces found:  1
[[15  6 62 62]]
image converted...
image created
0009_05289
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05289fr.jpg
Faces found:  1
[[16  6 63 63]]
image converted...
image created
0009_04935
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04935fr.jpg
Faces found:  1
[[29 10 64 64]]
image converted...
image created
0009_04970
Number of faces dete

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05815fr.jpg
Faces found:  1
[[ 4  9 61 61]]
image converted...
image created
0009_05043
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05043fr.jpg
Faces found:  1
[[40  8 65 65]]
image converted...
image created
0009_05168
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05168fr.jpg
Faces found:  1
[[35  7 67 67]]
image converted...
image created
0009_05298
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05298fr.jpg
Faces found:  1
[[22  9 64 64]]
image converted...
image created
0009_04451
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04451fr.jpg
Faces found:  1
[[41 12 68 68]]
image converted...
image created
0009_06042
Number of faces dete

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05292fr.jpg
Faces found:  1
[[19  8 61 61]]
image converted...
image created
0009_04949
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04949fr.jpg
Faces found:  1
[[22 10 66 66]]
image converted...
image created
0009_04462
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04462fr.jpg
Faces found:  1
[[32 11 73 73]]
image converted...
image created
0009_05389
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05389fr.jpg
Faces found:  1
[[31  5 59 59]]
image converted...
image created
0009_05925
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05925fr.jpg
Faces found:  1
[[13  8 68 68]]
image converted...
image created
0009_04979
Number of faces dete

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05184fr.jpg
Faces found:  1
[[27  7 65 65]]
image converted...
image created
0009_04660
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04660fr.jpg
Faces found:  1
[[34  9 71 71]]
image converted...
image created
0009_04281
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04281fr.jpg
Faces found:  1
[[27 12 74 74]]
image converted...
image created
0009_04471
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04471fr.jpg
Faces found:  1
[[23 10 72 72]]
image converted...
image created
0009_04238
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04238fr.jpg
Faces found:  1
[[29 12 72 72]]
image converted...
image created
0009_04220
Number of faces dete

Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05465fr.jpg
Faces found:  1
[[28 20 66 66]]
image converted...
image created
0009_04286
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04286fr.jpg
Faces found:  1
[[27 12 74 74]]
image converted...
image created
0009_05637
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05637fr.jpg
Faces found:  1
[[22  7 62 62]]
image converted...
image created
0009_05411
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_05411fr.jpg
Faces found:  1
[[30  8 68 68]]
image converted...
image created
0009_04318
Number of faces detected: 1
query image shape: (115, 115, 3)
/home/gonzalo/dev/post_data/MIT/test/0009/0009_04318fr.jpg
Faces found:  1
[[41 14 66 66]]
image converted...
image created
frontalization finished
